## Créer notre RNN pour faire la catégorisation des genres

In [1]:
%cd /tf/notebooks/notebooks
%pip install pandas
%pip install scikit-learn
%pip install seaborn

/tf/notebooks/notebooks

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

version = 3
df = pd.read_csv('clean_data/v{version}_audio_features_30s_equal.csv'.format(version=2))

In [3]:
# Encode labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Dictionnaire clef valeur du label_encoded:label
label_dict = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))

In [4]:
label_dict

{0: 'blues',
 1: 'classical',
 2: 'country',
 3: 'disco',
 4: 'hiphop',
 5: 'jazz',
 6: 'metal',
 7: 'pop',
 8: 'reggae',
 9: 'rock'}

In [5]:
from sklearn.preprocessing import StandardScaler

Y = df['label_encoded']
X = df.drop(['label', 'label_encoded'], axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)

# transform Y to a ndarray
Y = Y.to_numpy()

In [6]:
type(X)

numpy.ndarray

In [7]:
type(Y)

numpy.ndarray

In [8]:
Y

array([0, 0, 0, ..., 9, 9, 9])

In [9]:
from sklearn.model_selection import train_test_split


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

# extract X_val and Y_val from X_test and Y_test (0.5)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)

# save train, val and test into a csv
pd.DataFrame(X_train).to_csv('clean_data/v{version}_X_train.csv'.format(version=version), index=False)
pd.DataFrame(X_val).to_csv('clean_data/v{version}_X_val.csv'.format(version=version), index=False)
pd.DataFrame(X_test).to_csv('clean_data/v{version}_X_test.csv'.format(version=version), index=False)
pd.DataFrame(Y_train).to_csv('clean_data/v{version}_Y_train.csv'.format(version=version), index=False)
pd.DataFrame(Y_val).to_csv('clean_data/v{version}_Y_val.csv'.format(version=version), index=False)
pd.DataFrame(Y_test).to_csv('clean_data/v{version}_Y_test.csv'.format(version=version), index=False)

In [10]:
X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape

((930, 16), (310, 16), (310, 16), (930,), (310,), (310,))

In [11]:
# Reshape for LSTM RNN
from keras.utils import to_categorical

number_labels = len(label_dict)

# X_train_traited = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
# X_test_traited = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
# X_val_traited = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

X_train_traited = X_train
X_test_traited = X_test
X_val_traited = X_val

Y_train_traited = to_categorical(Y_train, num_classes=number_labels)
Y_test_traited = to_categorical(Y_test, num_classes=number_labels)
Y_val_traited = to_categorical(Y_val, num_classes=number_labels)

2023-11-16 11:39:36.961087: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 11:39:36.961137: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 11:39:36.961160: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 11:39:36.967894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
X_train_traited.shape, X_test_traited.shape, X_val_traited.shape, Y_train_traited.shape, Y_test_traited.shape, Y_val_traited.shape

((930, 16), (310, 16), (310, 16), (930, 10), (310, 10), (310, 10))

In [27]:
# Define RNN model
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Flatten
# model V1
# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(number_labels, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model V1
# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(number_labels, activation='softmax'))

# model V3
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(number_labels, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 128)               2176      
                                                                 
 dense_30 (Dense)            (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 flatten_2 (Flatten)         (None, 64)                0         
                                                                 
 dense_31 (Dense)            (None, 10)                650       
                                                                 
Total params: 11082 (43.29 KB)
Trainable params: 11082 (43.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [28]:
model.fit(X_train_traited, Y_train_traited, epochs=250, batch_size=32, validation_data=(X_val_traited, Y_val_traited), verbose=1)

Epoch 1/250
30/30 [==============================] - 1s 15ms/step - loss: 2.2071 - accuracy: 0.1753 - val_loss: 2.0158 - val_accuracy: 0.3065
Epoch 2/250
30/30 [==============================] - 0s 11ms/step - loss: 1.9371 - accuracy: 0.3366 - val_loss: 1.8294 - val_accuracy: 0.3613
Epoch 3/250
30/30 [==============================] - 0s 9ms/step - loss: 1.7659 - accuracy: 0.3903 - val_loss: 1.7031 - val_accuracy: 0.4258
Epoch 4/250
30/30 [==============================] - 0s 8ms/step - loss: 1.6402 - accuracy: 0.4409 - val_loss: 1.6152 - val_accuracy: 0.4452
Epoch 5/250
30/30 [==============================] - 0s 9ms/step - loss: 1.5731 - accuracy: 0.4710 - val_loss: 1.5703 - val_accuracy: 0.4839
Epoch 6/250
30/30 [==============================] - 0s 9ms/step - loss: 1.5211 - accuracy: 0.4645 - val_loss: 1.5296 - val_accuracy: 0.4871
Epoch 7/250
30/30 [==============================] - 0s 8ms/step - loss: 1.4763 - accuracy: 0.4860 - val_loss: 1.5090 - val_accuracy: 0.4935
Epoch 8/250

## Enregistrement du modèle

In [15]:
# save model
model.save('./models/v{version}_model.h5'.format(version=version))

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
loss, accuracy = model.evaluate(X_test_traited, Y_test_traited, verbose=1)
print(f'Accuracy model: {accuracy*100}%') 

10/10 [==============================] - 0s 11ms/step - loss: 2.0743 - accuracy: 0.6065
Accuracy model: 60.64516305923462%
